# Mean velocity of the Hyades cluster

The method was applied in Reino et al. (2018) consists of an iterative maximum-likelihood procedure 
to derive improved parallaxes for individual stars from their proper motions
and radial velocities by kinematically modelling the cluster.

Our method builds upon the maximum-likelihood method developed by Lindegren et al. (2000).
Whereas Lindegren et al. (2000), being interested in deriving astrometric radial velocities (see also Dravins et al. 1999), formulated their model in terms of proper motions as main observables, we generalised this to include measured, spectroscopic radial velocities:

1. We added radial velocity, whenever available, as fourth observable, besides trigonometric parallax and proper motion;

2. We made a transition from the $\chi^2$ statistic used in Lindegren et al. (2000), and denoted $g$, to a $p$ value or $1 − CDF(g, DOF)$ as a goodness-of-fit statistic;

3. We used a mixed three- and four-dimensional likelihood function so that both stars with and without known radial velocity can be treated simultaneously.

More details on the implementation of the method can be found in Reino et al. (2018), Section 5.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rc


from covmatrix import *
from minim_rv import *
rc('text', usetex = True)
rc('font', family = 'serif', weight = 'normal', size =15)

In [4]:
def read_gaia(filename):
    data = np.genfromtxt('members_rv.bin', delimiter  = ',', names = True)
    N = len(data)
    ID = np.array(data['source_id'], dtype= int)
    w = np.where(ID == 0)
    ID[w] = data['HIP'][w]

    deg2rad = np.pi/180.
    coords = np.zeros((N, 2))
    coords[:, 0] = data['ra']*deg2rad
    coords[:, 1] = data['dec']*deg2rad

    obs  = np.zeros((N, 3))
    obs[:, 0] = data['parallax']
    obs[:, 1] = data['pmra']
    obs[:, 2] = data['pmdec']


    err = np.zeros((N, 3))
    err[:, 0] = data['parallax_error']
    err[:, 1] = data['pmra_error']
    err[:, 2] = data['pmdec_error']

    vrad = data['RV']
    err_vrad = data['e_RV']

    corr = np.zeros((N, 3))
    corr[:, 0] = data['parallax_pmra_corr']
    corr[:, 1] = data['parallax_pmdec_corr']
    corr[:, 2] = data['pmra_pmdec_corr']

    bminv = data['BV']
    vmag = data['Vmag']
    
    
    return ID, coords, obs, err, vrad, err_vrad, corr, N

In [5]:
filename = 'members_rv.bin'
ID, coords, obs_astro, err_astro, vrad, err_vrad, corr, N = read_gaia(filename)

When the radial velocity is not available, RV = nan.

Initial guesses for the cartesian equatorial velocity and the dispersion for the **Hyades** cluster (units are km/s):

In [7]:
vx, vy, vz = -6.32,   45.24,  5.3 
sigmav = 0.3                      
initial_guesses = np.concatenate((obs_astro[:, 0], np.array([vx, vy, vz, sigmav])))

First iteration. This is the same as in the *standard* procedure (without radial velocities).

In [8]:
grad = 'YES'
method = 'Newton-CG'

result, nit = optimizer(grad, method, initial_guesses, coords[:, 0], coords[: ,1],
                        obs_astro[:,0], obs_astro[:,1], obs_astro[:,2], 
                        vrad, err_astro, err_vrad, 
                        corr, N)

Optimization terminated successfully.
         Current function value: 3868.103484
         Iterations: 22
         Function evaluations: 25
         Gradient evaluations: 380
         Hessian evaluations: 0


In [9]:
cl_fin = g_func(result, coords[:, 0], coords[:, 1], 
                obs_astro[:,0], obs_astro[:,1], obs_astro[:,2], 
                vrad, err_astro, err_vrad, corr, N)
cl_in =  g_func(initial_guesses, coords[:, 0], coords[:, 1], 
                obs_astro[:,0], obs_astro[:,1], obs_astro[:,2], 
                vrad, err_astro, err_vrad, corr, N)


In [10]:
H =  stella_Nmatrix_full(result, coords[:, 0], coords[:, 0], 
                         obs_astro[:,0], obs_astro[:,1], obs_astro[:,2], 
                         vrad, err_astro, err_vrad, corr, N)
invH =  np.linalg.inv(H)
err = np.zeros(N+4)
for i in range(N+4):
    err[i] = np.sqrt(invH[i,i])

print( 'vx: ',np.round(result[-4], 2), 'vy: ',np.round(result[-3],2),
      'vz: ', np.round(result[-2],2), 'sigma_v: ', np.round(result[-1], 2))

print( 'sigma_vx: ',np.round(err[-4], 2), 'sigma_vy: ',np.round(err[-3],2),
      'sigma_vz: ', np.round(err[-2],2), 'sigma_sigmav: ', np.round(err[-1], 2))

cl_fin_min = np.min(cl_fin)
print('Min CL value: ', np.round(cl_fin_min, 7))

vx:  -5.87 vy:  45.56 vz:  5.48 sigma_v:  0.96
sigma_vx:  0.07 sigma_vy:  0.09 sigma_vz:  0.1 sigma_sigmav:  0.03
Min CL value:  9.7e-06


In [12]:
cl_lim = 1-0.9973
while (cl_fin_min < cl_lim):
    N_survive = len(cl_fin[cl_fin > cl_fin_min])
    print('New N value: ', N_survive)

    ### Define new arrays with stars surviving the selection criterion
    coord_sel = np.zeros((N_survive, 2))
    astro_sel = np.zeros((N_survive, 3))
    vrad_sel = np.zeros(N_survive)
    err_sel, err_vrad_sel, corr_sel = np.zeros((N_survive, 3)), np.zeros(N_survive), np.zeros((N_survive, 3))
    ID_sel = np.zeros(N_survive)


    coord_sel[:, 0], coord_sel[:, 1] = coords[cl_fin > cl_fin_min, 0], coords[cl_fin > cl_fin_min, 1]
    astro_sel[:, 0], astro_sel[:, 1], astro_sel[:, 2] = obs_astro[cl_fin > cl_fin_min,0],\
                                                        obs_astro[cl_fin > cl_fin_min,1], \
                                                        obs_astro[cl_fin > cl_fin_min,2], 
    
    vrad_sel = vrad[cl_fin > cl_fin_min]
    
    err_sel[:, 0], err_sel[:, 1], err_sel[:, 2], err_vrad_sel = err_astro[cl_fin > cl_fin_min,0], \
                                                                err_astro[cl_fin > cl_fin_min,1],\
                                                                err_astro[cl_fin > cl_fin_min,2], \
                                                                err_vrad[cl_fin > cl_fin_min]
    corr_sel[:, 0], corr_sel[:, 1], corr_sel[:, 2] = corr[cl_fin > cl_fin_min,0], corr[cl_fin > cl_fin_min,1], \
                                                     corr[cl_fin > cl_fin_min,2] 
    
    ID_sel = ID[cl_fin > cl_fin_min]

    ID_reject = ID[(cl_fin <= cl_fin_min)][0]
    print('ID of excluded star: ', ID[(cl_fin <= cl_fin_min)])

    
    ### Minimize likelihood
    N = N_survive
    initial_guesses_sel =  np.concatenate((astro_sel[:, 0], np.array([vx, vy, vz, sigmav])))
    result, nit_sel = optimizer(grad, method, initial_guesses_sel, coord_sel[:, 0], coord_sel[:, 1], 
                                astro_sel[:, 0], astro_sel[:, 1], astro_sel[:, 2], vrad_sel, 
                                err_sel, err_vrad_sel, corr_sel, N)
    cl_fin = g_func(result, coord_sel[:, 0], coord_sel[:, 1], 
                                astro_sel[:, 0], astro_sel[:, 1], astro_sel[:, 2], vrad_sel, 
                                err_sel, err_vrad_sel, corr_sel, N)

    cl_fin_min = np.min(cl_fin)


    ### Compute errors on quantities.
    H =  stella_Nmatrix_full(result, coord_sel[:,0], 
                             coord_sel[:,0], astro_sel[:, 0], astro_sel[:, 1], astro_sel[:, 2],
                             vrad_sel, err_sel, err_vrad_sel, corr_sel, N)
    invH =  np.linalg.inv(H)

    err_param = np.zeros(N+4)
    for i in range(N+4):
        err_param[i] = np.sqrt(invH[i,i])

    ### Upload initial values
    coords = np.zeros((N, 2))
    obs_astro, err_astro, corr  = np.zeros((N, 3)), np.zeros((N, 3)), np.zeros((N, 3))
    coords[:, 0], coords[:,1] = coord_sel[: ,0], coord_sel[:, 1]
    obs_astro[:, 0], obs_astro[:, 1], obs_astro[:, 2] = astro_sel[:, 0], astro_sel[:, 1], astro_sel[:, 2]
    vrad = vrad_sel
    err_astro[:, 0], err_astro[:, 1], err_astro[:, 2], err_vrad = err_sel[:,0], err_sel[:,1], err_sel[:,2], err_vrad_sel
    corr[:, 0], corr[:, 1], corr[:, 2] = corr_sel[:, 0], corr_sel[:, 1], corr_sel[:, 2]
    ID = ID_sel


    U =  Ulike(result, coord_sel[:, 0], coord_sel[:, 1],
               astro_sel[:, 0],astro_sel[:, 1],astro_sel[:, 2], 
               vrad_sel, err_sel, err_vrad_sel, corr_sel, N)
    
    ### Print 
    print( 'Min CL value: ', cl_fin_min)
    print( 'Likelihood: ', U)
    print( 'v_x: ',result[-4], 'v_y: ',result[-3], 'v_z: ', result[-2], 'sigma_v: ', result[-1])

              

New N value:  250
ID of excluded star:  [3309967055477808128]
Optimization terminated successfully.
         Current function value: 3828.221487
         Iterations: 20
         Function evaluations: 24
         Gradient evaluations: 281
         Hessian evaluations: 0
Min CL value:  9.722496908e-06
Likelihood:  3828.22148662
v_x:  -5.8653942022 v_y:  45.5911702843 v_z:  5.48506762717 sigma_v:  0.942369924439
New N value:  249
ID of excluded star:  [38453460676522752]
Optimization terminated successfully.
         Current function value: 3797.021298
         Iterations: 21
         Function evaluations: 25
         Gradient evaluations: 309
         Hessian evaluations: 0
Min CL value:  7.23598500551e-06
Likelihood:  3797.02129757
v_x:  -5.88276993433 v_y:  45.6072803501 v_z:  5.48075254748 sigma_v:  0.919209174418
New N value:  248
ID of excluded star:  [124145479935111424]
Optimization terminated successfully.
         Current function value: 3764.976093
         Iterations: 21
     

Optimization terminated successfully.
         Current function value: 3193.140915
         Iterations: 22
         Function evaluations: 25
         Gradient evaluations: 350
         Hessian evaluations: 0
Min CL value:  6.0299090438e-05
Likelihood:  3193.14091468
v_x:  -5.99450886386 v_y:  45.5876782752 v_z:  5.48421947282 sigma_v:  0.595656459309
New N value:  229
ID of excluded star:  [3306066434899058688]
Optimization terminated successfully.
         Current function value: 3167.296938
         Iterations: 24
         Function evaluations: 27
         Gradient evaluations: 494
         Hessian evaluations: 0
Min CL value:  4.51725597652e-05
Likelihood:  3167.29693821
v_x:  -5.99857646158 v_y:  45.5920535672 v_z:  5.49607639305 sigma_v:  0.575953784582
New N value:  228
ID of excluded star:  [114317976286533888]
Optimization terminated successfully.
         Current function value: 3141.476651
         Iterations: 14
         Function evaluations: 19
         Gradient evaluations

Min CL value:  0.000168226752238
Likelihood:  2568.4458356
v_x:  -5.99875623071 v_y:  45.5805775543 v_z:  5.54435686224 sigma_v:  0.374553269311
New N value:  209
ID of excluded star:  [20205]
         Current function value: 3252.853510
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 8537
         Hessian evaluations: 0
Min CL value:  1.76730775938e-29
Likelihood:  3252.8535097
v_x:  -6.27233960196 v_y:  45.2557977259 v_z:  5.34219713505 sigma_v:  0.896606509876
New N value:  208
ID of excluded star:  [21092]
         Current function value: 3103.625857
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 8497
         Hessian evaluations: 0
Min CL value:  1.44743918716e-09
Likelihood:  3103.62585731
v_x:  -6.26960734593 v_y:  45.2541989739 v_z:  5.34251157089 sigma_v:  0.886235555411
New N value:  207
ID of excluded star:  [20873]
Optimization terminated successfully.
         Current function value: 2491.343721

Optimization terminated successfully.
         Current function value: 2019.549362
         Iterations: 21
         Function evaluations: 28
         Gradient evaluations: 366
         Hessian evaluations: 0
Min CL value:  0.00140410977935
Likelihood:  2019.54936203
v_x:  -5.94035200312 v_y:  45.5995067144 v_z:  5.5777117424 sigma_v:  0.254512388182
New N value:  188
ID of excluded star:  [21029]
Optimization terminated successfully.
         Current function value: 1996.814185
         Iterations: 18
         Function evaluations: 26
         Gradient evaluations: 369
         Hessian evaluations: 0
Min CL value:  0.00170774228311
Likelihood:  1996.81418535
v_x:  -5.95078942908 v_y:  45.6042673084 v_z:  5.57314553941 sigma_v:  0.251760932827
New N value:  187
ID of excluded star:  [20916]
Optimization terminated successfully.
         Current function value: 1972.067521
         Iterations: 20
         Function evaluations: 26
         Gradient evaluations: 2603
         Hessian evalu

In [18]:
print('sigma_v_x:', np.round(err_param[-4], 2))
print('sigma_v_y:', np.round(err_param[-3], 2))
print('sigma_v_z:', np.round(err_param[-2], 2))
print('sigma_sigmav', np.round(err_param[-1], 2))

sigma_v_x: 0.03
sigma_v_y: 0.06
sigma_v_z: 0.07
sigma_sigmav 0.01
